<a href="https://colab.research.google.com/gist/MateuszCzz/afc29fdca891ea324e10fbc0af67c3e2/osuphoenix-io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReadMe:<br>


> if running first time:
>> go from top to bottom
>> except skip the loadings

> if not: 
>> always run step 0 and then prociede from the right load 
 
:) gl 
function for json editor:
<br>
function query(data) {
  return _.chain(data)
    .map(obj => {
      if (obj.length >= 9) {
        return obj[2];
      } else if (obj.length >= 8) {
        return obj[1];
      } else {
        return null;
      }
    })
    .compact()
    .uniq()
    .value();
}

<br> https://www.convertjson.com/html-table-to-json.htm
<br> https://jsoneditoronline.org/#right=local.qekilo&left=local.vuciri


# Step 0:
## imports, config

In [ ]:
!pip install ossapi
import csv
import os
import pickle
import re
from collections import defaultdict
from google.colab import drive, files, output
from ossapi import Ossapi
from prettytable import PrettyTable
import operator
import statistics


# Mount your Google Drive
drive.mount('/content/drive')

#clean
output.clear()

#configuration
min_pp = 150 #nickname scores will take half of this
nickname = "Reset_676"
nickname_rank = 7799195

## PUT IN YOUR OWN
api_number = 123
api_key = "###"
api = Ossapi(api_number, api_key)
mode_chosen = "osu"
ranking_type = "performance"
bot_cap = 165000
top_cap = 235000

# Countries in top 50 except us
country_codes = [
                 "PL", "DE", "CA", "PH", "JP", "FR", "BR", "GB", "ID", "AU", "TW", "CL", "MY",
                 "KR", "UA", "CN", "MX", "IT", "HK", "FI", "ES", "NL", "SE", "TR", "CZ", "RO",
                 "PT", "NO", "AT", "CO", "HU", "KZ", "BE", "BY", "NZ", "LT", "DK", "IN", "GR" 
                ]

# Functions for later
def filter_scores(user_ids, api, min_pp):
    osu_mode_scores = []
    counter = 0 
    for user_id in user_ids:
        if(counter%100==0):
          print(counter)
        counter+=1
        user_scores = api.user_scores(user_id, "best", limit=150, mode=mode_chosen)
        user_scores = [score for score in user_scores if score.pp >= min_pp and score.mode.value == mode_chosen]
        osu_mode_scores.extend(user_scores)
    return osu_mode_scores

# Step 1.1: 
## get IDs of players

In [ ]:
# Variable to hold the end result
all_ids = []

# Loop through countries
for country_code in country_codes:
    cursor = None
    # Keep requesting until no more left
    while True:
        response = api.ranking(mode=mode_chosen, type=ranking_type,
                               country=country_code, cursor=cursor)
        # Add the new rankings
        all_ids += response.ranking
        cursor = response.cursor
        if cursor is None:
            break
    print(f"Completed: {country_code} requests")

# Print the number of rankings received
print(f"Received {len(all_ids)} rankings")

In [5]:
# Filter them 
userlist = [player.user.id  for player in all_ids if bot_cap <= player.global_rank <= top_cap]

# Print number after filter received
print(f"After Filter: {len(all_ids)} rankings")

# Split the filtered_rankings
num_users = len(userlist)
part_size = num_users // 10
user_ids_part1 = userlist[:part_size]
user_ids_part2 = userlist[part_size:2*part_size]
user_ids_part3 = userlist[2*part_size:3*part_size]
user_ids_part4 = userlist[3*part_size:4*part_size]
user_ids_part5 = userlist[4*part_size:5*part_size]
user_ids_part6 = userlist[5*part_size:6*part_size]
user_ids_part7 = userlist[6*part_size:7*part_size]
user_ids_part8 = userlist[7*part_size:8*part_size]
user_ids_part9 = userlist[8*part_size:9*part_size]
user_ids_part10 = userlist[9*part_size:]

# Step 1.2:
  save the data for future use

In [ ]:
# Save to Google Drive
with open('/content/drive/MyDrive/user_ids.pkl', 'wb') as f:
    pickle.dump((user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10), f)

# Step 1.3:
  load the data from earlier 

In [ ]:
# Load from the Google Drive
with open('/content/drive/MyDrive/user_ids.pkl', 'rb') as f:
    user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)

# Step 2.1:
  first part

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part1 = filter_scores(user_ids_part1, api, min_pp)
osu_mode_scores_part2 = []
osu_mode_scores_part3 = []
osu_mode_scores_part4 = []
osu_mode_scores_part5 = []
osu_mode_scores_part6 = []
osu_mode_scores_part7 = []
osu_mode_scores_part8 = []
osu_mode_scores_part9 = []
osu_mode_scores_part10 = []

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.2:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part2 = filter_scores(user_ids_part2, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.3:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part3 = filter_scores(user_ids_part3, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.4:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part4 = filter_scores(user_ids_part4, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.5:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part5 = filter_scores(user_ids_part5, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.6:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part6 = filter_scores(user_ids_part6, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.7:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part7 = filter_scores(user_ids_part7, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.8:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part8 = filter_scores(user_ids_part8, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.9:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part9 = filter_scores(user_ids_part9, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.10:

In [ ]:
# Declarations for 
# Filter scores by pp and mode for each part
osu_mode_scores_part10 = filter_scores(user_ids_part10, api, min_pp)

# Save
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'wb') as f:
    pickle.dump((osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10), f)

## Load:


In [ ]:
# Load user IDs if not set already
if 'user_ids_part1' not in locals():
    file_path = '/content/drive/MyDrive/user_ids.pkl'
    with open(file_path, 'rb') as f:
        user_ids_part1, user_ids_part2, user_ids_part3, user_ids_part4, user_ids_part5, user_ids_part6, user_ids_part7, user_ids_part8, user_ids_part9, user_ids_part10 = pickle.load(f)
else:
    print("ids set already!")

# Load the filtered scores from a file on Google Drive
with open('/content/drive/MyDrive/osu_mode_scores.pkl', 'rb') as f:
    osu_mode_scores_part1, osu_mode_scores_part2, osu_mode_scores_part3, osu_mode_scores_part4, osu_mode_scores_part5, osu_mode_scores_part6, osu_mode_scores_part7, osu_mode_scores_part8, osu_mode_scores_part9, osu_mode_scores_part10 = pickle.load(f)

# Step 2.11 
Integration

In [ ]:
# Combine scores from all parts
scores = []
scores.extend(osu_mode_scores_part1)
scores.extend(osu_mode_scores_part2)
scores.extend(osu_mode_scores_part3)
scores.extend(osu_mode_scores_part4)
scores.extend(osu_mode_scores_part5)
scores.extend(osu_mode_scores_part6)
scores.extend(osu_mode_scores_part7)
scores.extend(osu_mode_scores_part8)
scores.extend(osu_mode_scores_part9)
scores.extend(osu_mode_scores_part10)

# Step 3:
User

In [ ]:
# Setting up the user
nicknameScores =  api.user_scores(nickname_rank, "best", limit=500, mode="osu")
nicknameScores = [score for score in nicknameScores if score.pp >= min_pp/2]

# Step 4:
Data formation

In [ ]:
# Group by beatmap, mods
scores_by_beatmap_mods = defaultdict(list)
for score in scores:
    beatmap_mods_str = str(score.beatmap.id) + '\t' + str(score.mods)
    scores_by_beatmap_mods[beatmap_mods_str].append(score)

# Calc: num of scores, avg pp, avg combo, avg acc, avg fc
beatmap_stats = {}
for beatmap_mods_str, beatmap_scores in scores_by_beatmap_mods.items():
    beatmap_id, mods = beatmap_mods_str.split('\t')
    beatmap_name = beatmap_scores[0].beatmapset.title
    beatmap_creator = beatmap_scores[0].beatmapset.creator
    beatmap_link = "https://osu.ppy.sh/beatmaps/" + str(beatmap_id)
    num_scores = len(beatmap_scores)
    num_perfect_scores = sum(score.perfect for score in beatmap_scores)
    total_pp = sum(score.pp for score in beatmap_scores)
    avg_pp = total_pp / num_scores if num_scores > 0 else 0
    total_max_combo = sum(score.max_combo for score in beatmap_scores)
    avg_max_combo = total_max_combo / num_scores if num_scores > 0 else 0
    total_accuracy = sum(score.accuracy for score in beatmap_scores)
    avg_accuracy = total_accuracy / num_scores if num_scores > 0 else 0
    hit_length = beatmap_scores[0].beatmap.hit_length
    percentage_perfect = num_perfect_scores / num_scores * 100 if num_scores > 0 else 0
    beatmap_stats[beatmap_mods_str] = {'beatmap_name': beatmap_name,
                                       'beatmap_link': beatmap_link,
                                       'num_scores': num_scores,
                                       'num_perfect_scores': num_perfect_scores,
                                       'percentage_perfect': percentage_perfect,
                                       'avg_pp': avg_pp,
                                       'avg_max_combo': avg_max_combo,
                                       'avg_accuracy': avg_accuracy,
                                       'length': hit_length,
                                       'mods': mods,
                                       'beatmap_creator': beatmap_creator}


# Step 5:
tables

# table for farming

In [ ]:
#config
a = 0.1 # factor of pp 
b = 1   # factor of number of scores 

# Print the data using prettytables
# table = PrettyTable(border=False)
table = PrettyTable()
table.field_names = ["Beatmap Name", "Beatmap Creator", "Mods","length", "Avg PP", "Avg Combo", "Avg Acc", "Number of Scores", "FC %", "Link", "Score"]

# sort by decreasing value of (a*pp + b*occurrences)
key_func = lambda x: -(a * x[1]['avg_pp'] + b * x[1]['num_scores'])

for beatmap_key, beatmap_data in sorted(beatmap_stats.items(), key=key_func):
    # extract the beatmap id and mods string from the key
    beatmap_id, mods_str = beatmap_key.split("\t")
    mods = mods_str.split(",")
    
    beatmap_name = beatmap_data['beatmap_name']
    beatmap_creator = beatmap_data['beatmap_creator']
    num_scores = beatmap_data['num_scores']
    length = beatmap_data['length']
    avg_pp = beatmap_data['avg_pp']
    avg_max_combo = beatmap_data['avg_max_combo']
    avg_accuracy = beatmap_data['avg_accuracy']
    percentage_perfect = beatmap_data['percentage_perfect']
    score = a * avg_pp + b * num_scores
    link = beatmap_data['beatmap_link']
    table.add_row([beatmap_name, beatmap_creator, ", ".join(mods),length, round(avg_pp, 2), round(avg_max_combo, 2), round(avg_accuracy, 2), num_scores, f"{round(percentage_perfect, 2)}%", link, round(score, 2)])

table.align["Beatmap Name"] = "l"
table.align["Beatmap Creator"] = "l"
table.align["Mods"] = "m"
print(table)


In [ ]:
output_name = "table.txt"
# Replace 3 or more spaces in a row with a comma
table_txt =  table.get_string()
# Export table to CSV
with open(output_name, 'w') as file:
    file.write(table_txt)
# Download CSV file
files.download(output_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# table wit user

In [ ]:
# Filter parameters
min_occurrence = 1
min_max_combo = 1
min_score_difference = -2000  # Minimum allowed score difference in pp
max_score_difference =  2000  # Maximum allowed score difference in pp    

# Print the data using prettytables
table = PrettyTable()
table.field_names = ["Beatmap Name", "Creator", "Length", "Mods", nickname + "'s PP", "Avg PP", "Avg Max Combo", "Avg Acc", "FC %", "Number of Scores", "Link", "Score Difference"]

for beatmap_id, beatmap_data in beatmap_stats.items():
    num_scores = beatmap_data['num_scores']
    avg_pp = beatmap_data['avg_pp']
    avg_max_combo = beatmap_data['avg_max_combo']
    avg_accuracy = beatmap_data['avg_accuracy']
    perfect_percentage = beatmap_data['percentage_perfect']
    beatmap_name = beatmap_data['beatmap_name']
    link = beatmap_data['beatmap_link']
    creator = beatmap_data['beatmap_creator']
    length = beatmap_data['length']
    mods = beatmap_data['mods']

    # Check if the beatmap meets the filter criteria
    if num_scores < min_occurrence or avg_max_combo < min_max_combo:
        continue

    # Check if the beatmap is in the user's scores
    user_score = next((score for score in nicknameScores if (str(score.beatmap.id) + '\t' + str(score.mods)) == beatmap_id), None)
    if user_score is None:
        nickname_pp = 0
    else:
        nickname_pp = user_score.pp
    if nickname_pp == 0:
        score_difference = 0
    else:
        score_difference = nickname_pp - avg_pp

    # Check if the score difference is within the allowed range
    if (score_difference) < (min_score_difference) or (score_difference) > (max_score_difference):
        continue

    table.add_row([beatmap_name, creator, length, mods, nickname_pp, round(avg_pp, 2), round(avg_max_combo, 2), round(avg_accuracy, 2), format(round(perfect_percentage, 2), ".2f") + "%", num_scores, link, format(round(score_difference, 2), ".2f")+ " pp"])

# Set the alignment of the first column to left
table.align["Beatmap Name"] = "l"
table.align["Link"] = "l"
table.align["Score Difference"] = "r"

# Print the table
print(table.get_string(sortby="Number of Scores", reversesort=True))


In [ ]:
output_name = 'table of '+ nickname + '.txt'

# Replace 3 or more spaces in a row with a comma
table_txt =  table.get_string()
# Export table to CSV
with open(output_name, 'w') as file:
    file.write(table_txt)
# Download CSV file
files.download(output_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>